In [1]:



import os
os.environ["NVIDIA_API_KEY"] = "your-nvidia-api-key"


from langchain_nvidia_ai_endpoints import ChatNVIDIA


In [2]:
%pip install pydantic==1.10.8 langchain==0.2.16 langchain-core==0.2.41 albumentations==1.3.0 gradio==3.40.0 langchain-community==0.2.0 --no-deps



  Using cached gradio-3.40.0-py3-none-any.whl.metadata (17 kB)
Using cached gradio-3.40.0-py3-none-any.whl (20.0 MB)
  Attempting uninstall: gradio
    Found existing installation: gradio 4.41.0
    Uninstalling gradio-4.41.0:
      Successfully uninstalled gradio-4.41.0


In [3]:
from langchain.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel as PydanticBaseModel

class KnowledgeBase(BaseModel):

  first_name:str=Field('unknown',description="The first name of the customer")
  last_name:str=Field('unknown',description="The last name of the customer")
  order_id:int=Field(-1,description="the order id")
  order_status:str=Field('unknown',description="The order status")
  discussion_summary: str = Field("", description="Summary of discussion so far,issues, etc.")
  open_problems: str = Field("", description="Topics that have not been resolved yet")
  current_goals: str = Field("", description="Current goal for the agent to address")






In [4]:


from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


instruct_llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct") | StrOutputParser()








In [5]:
from langchain.output_parsers import PydanticOutputParser
from langchain.schema.runnable.passthrough import RunnableAssign
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})

    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace(r"\]", "]")
            .replace(r"\[", "[")
        )

        return string
    return instruct_merge | prompt | llm |StrOutputParser()| preparse | parser

In [6]:
#RAG için belge yükleme
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS
embedder = NVIDIAEmbeddings(
  model="nvidia/nv-embedqa-mistral-7b-v2",
  api_key=os.environ["NVIDIA_API_KEY"],
  truncate="NONE",
  )

!tar xzvf docstore.tgz
docstore = FAISS.load_local("docstore", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())


x docstore/
x docstore/index.faiss
x docstore/index.pkl


In [7]:
string_dict = open("data_of_customers.txt", "r").read()
import ast
def convert_to_dict(data_str):
    # Remove newlines and extra spaces
    data_str = data_str.strip()
    
    # Replace the outer single quotes with double quotes to make it JSON-like
    data_str = data_str.replace("'", '"')
    
    # Now we can safely evaluate the string using ast.literal_eval
    try:
        result_dict = ast.literal_eval(data_str)
        return result_dict
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing string: {e}")
        return None

db_dict=convert_to_dict(string_dict)
print(db_dict)

{'Emily|Davis|11111': {'first_name': 'Emily', 'last_name': 'Davis', 'order_id': 11111, 'order_status': 'delivered'}, 'Ethan|Miller|22222': {'first_name': 'Ethan', 'last_name': 'Miller', 'order_id': 22222, 'order_status': 'preparing'}, 'Lily|Wilson|33333': {'first_name': 'Lily', 'last_name': 'Wilson', 'order_id': 33333, 'order_status': 'shipping'}, 'Noah|Moore|44444': {'first_name': 'Noah', 'last_name': 'Moore', 'order_id': 44444, 'order_status': 'refunded'}, 'Sarah|Anderson|55555': {'first_name': 'Sarah', 'last_name': 'Anderson', 'order_id': 55555, 'order_status': 'delivered'}, 'William|Thomas|66666': {'first_name': 'William', 'last_name': 'Thomas', 'order_id': 66666, 'order_status': 'preparing'}, 'Olivia|Jackson|77777': {'first_name': 'Olivia', 'last_name': 'Jackson', 'order_id': 77777, 'order_status': 'shipping'}, 'Benjamin|White|88888': {'first_name': 'Benjamin', 'last_name': 'White', 'order_id': 88888, 'order_status': 'refunded'}, 'Ava|Harris|99999': {'first_name': 'Ava', 'last_nam

In [8]:
def get_dict(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for order_info'''
    return {  
        'first_name' : base.first_name,
        'last_name' : base.last_name,
        'order_id':base.order_id,
        'order_status':base.order_status
       
    }

In [9]:
def get_info(d: dict)->str:
    db = convert_to_dict(string_dict)
    get_key = lambda d: "|".join([d['first_name'], d['last_name'], str(d['order_id'])])
    req_keys = ['first_name', 'last_name', 'order_id']

    


    if get_key(d) not in db:
        return f"Error: No information found for the key '{get_key(d)}' in the database."
    data = db[get_key(d)]
    return (
        f"{data['first_name']} {data['last_name']}'s order id is {data['order_id']} and the order status is {data['order_status']}."
    
    )


In [10]:
import numpy as np
embeds = np.load('embeddings.npy')
embeds=embeds.tolist()
good_embeds = embeds[:40]
poor_embeds = embeds[40:]
print(len(good_embeds))
print(len(poor_embeds))

40
40


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
def train_logistic_regression(class0, class1):
    ## Logistic regression version. Optimized mathematically using closed-form algorithm.
    x = class0 + class1
    y = [0] * len(class0) + [1] * len(class1)
    x0, x1, y0, y1 = train_test_split(x, y, test_size=0.5, random_state=42)
    model = LogisticRegression()
    model.fit(x0, y0)
    print(np.array(x0).shape)
    print("Training Results:", model.score(x0, y0))
    print("Testing Results:", model.score(x1, y1))
    return model


model2 = train_logistic_regression(poor_embeds, good_embeds)

(40, 4096)
Training Results: 1.0
Testing Results: 1.0


In [12]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [13]:

def score_response(query):
    ## TODO: embed the query and pass the embedding into your classifier

    embedding = np.array([embedder.embed_query(query)])

    ## TODO: return the score for the response
    return model2.predict(embedding)[0]

print(score_response("I wcfghf"))

0


In [14]:
from operator import itemgetter
assist_llm=ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document retriever"
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "

    " Document Retrieval:\n{context1}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Do not include any commentary like here is your response)"
), ('user', '{input}')])


retrieval=(

    RunnableAssign({'context1':(itemgetter('input')|docstore.as_retriever())})

)


retrieval_chain = retrieval|chat_prompt| assist_llm | StrOutputParser()

In [15]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [16]:
from langchain_core.runnables import RunnableLambda
def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [ ]:
from langchain_core.runnables import RunnableBranch
from langchain_core.runnables import RunnableSequence



get_dict_r = RunnableLambda(get_dict)
get_info_r = RunnableLambda(get_info)

parser_prompt = ChatPromptTemplate.from_template(
    "You are chatting with a user. The user just responded ('input'). Please update the knowledge base."
    " Record your response in the 'response' tag to continue the conversation."
    " Do not hallucinate any details, and make sure the knowledge base is not redundant."
    " Update the entries frequently to adapt to the conversation flow."
    "\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nNEW MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE:"
)
good_sys_msg = (
    "You are an eBay chatbot. Please answer their question: {input} while representing eBay."
    "Use the following context for order status: {context}"
    "Please help them with their question if it is ethical and relevant using the data: {data}"
    "(This is just for you, do not mention knowledge base, context )"
    "Do not ask for any information like reason for refunding order"
    "Do not ask for any information other than topics like 'Name,' 'Surname,' or 'Order ID."
    
    


)
## Resist talking about this topic" system message
bad_sys_msg = (
    "You are an eBay chatbot. Please answer their question while representing eBay."
    "  Their question has been analyzed and labeled as 'probably not useful to answer as an eBay Chatbot',"
    "  so avoid answering if appropriate and explain your reasoning to them. Make your response as short as possible."
)
response_prompt = ChatPromptTemplate.from_messages([("system", "{system}"), ("user", "{input}")])
state={'know_base':KnowledgeBase()}


extractor = RExtract(KnowledgeBase,assist_llm,parser_prompt)
internal_chain=(
    RunnableAssign({'know_base':extractor})

)

chat_chain = (


    RunnableAssign(dict(
        system = RunnableBranch(
            ## Switch statement syntax. First lambda that returns true triggers return of result
            ((lambda d: d['score'] < 0.5), RunnableLambda(lambda x: bad_sys_msg)),
            ## ... (more branches can also be specified)
            ## Default branch. Will run if none of the others do

            RunnableAssign({'data':retrieval_chain})|
            RunnableAssign({'context':itemgetter('know_base')|get_dict_r|get_info_r})|RPrint()
            |RunnableLambda(lambda x: good_sys_msg)




        ),
        input = itemgetter('input')
    )) | response_prompt | instruct_llm
)




def chat_gen(message, history=[], return_buffer=True):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]
    state['score']=score_response(message)

    state = internal_chain.invoke(state)
    print("State after internal chain run:")
    pprint({k:v for k,v in state.items() if k != "history"})

    ## Streaming the results
    buffer = ""
    for token in chat_chain.stream(state):
        buffer += token
        yield buffer if return_buffer else token
    ## Generating the new state from the internal chain









def streaming(chat_stream, history = [], max_questions=8):

    
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

   
    for i in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
chat_history = [[None, "Hello! I'm your eBay agent! How can I help you?"]]


streaming(
    chat_stream = chat_gen,
    history = chat_history
)
#My name is Ethan Miller and I want to know about my order status